In [ ]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

classes=['0','1','2','3','4','5','6','7','8','9']
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1,))])

root='./data'
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)


batch_size=150

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
shuffle=True)

test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
shuffle=False)

class LSTM(nn.Module):
    
    def __init__(self):
        
        super(LSTM,self).__init__()
        
        self.lstm=nn.LSTM(28,128,1)
        self.fc1=nn.Linear(28*128,128)
        
        self.bn1=nn.BatchNorm1d(128)
        self.fc3=nn.Linear(128,64)
        self.fc2=nn.Linear(64,10)
        
    def forward(self,x):
        output,(h_n,c_n)=self.lstm(x,None)
        g=output
        g=g.view(-1,28*128)
        
        g=F.relu(self.fc1(g))
        
        g=self.bn1(g)
        g=F.relu(self.fc3(g))
        g=F.relu(self.fc2(g))
        
        return g

    
lstm=LSTM()
lstm.cuda()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)
criteria= nn.CrossEntropyLoss()
for epoch in range(30):
    ave_loss=0.0
    for i,data in enumerate(train_loader,0):
        inputs,labels=data
        inputs=inputs.view(-1,28,28)
        inputs=Variable(inputs.cuda())
        #print(inputs.shape)
        #print(inputs.shape)
        optimizer.zero_grad()
        output=lstm(inputs)
        labels=Variable(labels.cuda())
        #output=Variable(output.cuda())
        loss=criteria(output,labels)
        loss.backward()
        optimizer.step()
        ave_loss+=loss.item()
        if i%100==99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 100))
            ave_loss = 0.0
correct=0
total=0
with torch.no_grad():
    for data in test_loader:
        images,labels=data
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())
        images=images.view(-1,28,28)
        outputs=lstm(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

print('Accuracy of the network on the  test images: %0.6f %%' % (
    100 * correct / total))

[1,   100] loss: 1.085
[1,   200] loss: 0.881
[1,   300] loss: 0.863
[1,   400] loss: 0.849
[2,   100] loss: 0.805
[2,   200] loss: 0.818
[2,   300] loss: 0.798
[2,   400] loss: 0.800
[3,   100] loss: 0.775
[3,   200] loss: 0.790
[3,   300] loss: 0.774
[3,   400] loss: 0.786
[4,   100] loss: 0.762
[4,   200] loss: 0.770
[4,   300] loss: 0.767
[4,   400] loss: 0.762
[5,   100] loss: 0.748
[5,   200] loss: 0.761
[5,   300] loss: 0.751
[5,   400] loss: 0.773
[6,   100] loss: 0.752
[6,   200] loss: 0.756
[6,   300] loss: 0.737
[6,   400] loss: 0.753
[7,   100] loss: 0.737
[7,   200] loss: 0.758
[7,   300] loss: 0.740
[7,   400] loss: 0.740
[8,   100] loss: 0.745
[8,   200] loss: 0.746
[8,   300] loss: 0.741
[8,   400] loss: 0.727
